# Зависимости

In [106]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (Embedding, Conv1D, MaxPooling1D,
                                     GlobalAveragePooling1D, Dropout,
                                     Dense, LSTM)
from tensorflow.keras import utils
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import files

from mycsv import split_file, shuffle_file_lines
from dataset import load_dataset
from transform import Converter

# Загрузка и конвертация данных

Перемешка функций в датасете.

In [102]:
shuffle_file_lines('dataset.csv', new_file=True)

Разделение датасета на тренировочную выборку и тестовую.

In [103]:
split_file('shuffled_dataset.csv', split_ratio=0.8)

После разделения датасета в дериктории появиться два файла (тренировочный сет и тестовый).

* `maxlen` задает максимальную длину последовательности узлов функции (средняя длина последовательности ~50)

* `verbose` задает многословность функций обработчиков (то столько будет выводится информации во время обратоки); доступные значения от 0 до 3

In [104]:
train_file_name = 'train_shuffled_dataset.csv'
test_file_name = 'test_shuffled_dataset.csv'
maxlen = 250

Загрузка и трансформация данных для обучения.

In [ ]:
dataset = load_dataset(train_file_name, test_file_name, maxlen, verbose=1)
x_train, y_train, x_test, y_test, data_converter, marks_converter = dataset

Сериализация словарей конвертеров.

In [109]:
data_converter.to_file()
marks_converter.to_file()

# Выбор модели нейронной сети

In [110]:
# CNN-Model
model_name = 'fnp_cnn'
model = Sequential()
model.add(Embedding(len(data_converter), 32, input_length=maxlen))
model.add(Conv1D(100, 10, activation='relu', padding='same'))
model.add(Conv1D(100, 10, activation='relu', padding='same'))
model.add(MaxPooling1D(3))
model.add(Conv1D(160, 10, activation='relu', padding='same'))
model.add(Conv1D(160, 10, activation='relu', padding='same'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(len(marks_converter), activation='softmax'))

In [116]:
# LSTM-Model
model_name = 'fnp_lstm'
model = Sequential()
model.add(Embedding(len(data_converter), 32, input_length=maxlen))
model.add(LSTM(128, return_sequences=True, dropout=0.2))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(len(marks_converter), activation='softmax'))

In [ ]:
print('Название модели:', model_name)
model.summary()

# Обучение нейронной сети

In [118]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
graph_history = {'accuracy': [], 'val_accuracy': []}

In [ ]:
history = model.fit( x_train, y_train,
                            validation_data = (x_test, y_test),
                            epochs=10,
                            batch_size=100,
                            shuffle=True,
                            verbose=1)
graph_history['accuracy'].extend(history.history['accuracy'])
graph_history['val_accuracy'].extend(history.history['val_accuracy'])

# Проверка качества обучения

In [ ]:
plt.plot(graph_history['accuracy'],
         label='Доля верных ответов на обучающем наборе')
plt.plot(graph_history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Доля верных ответов на тестовых данных составляет:',round(scores[1]* 100, 4),'%')

# Сохранить/загрузить модель нейронной сети

Загрузка модели нейронной сети из файла.

In [ ]:
MODEL_PATH = ''
model = load_model(MODEL_PATH)

Сохранение нейронной сети в директорию.

In [125]:
model.save('{name}_maxlen{len}_e{epochs}_p{precent}.h5'.format(
    name=model_name,
    len=maxlen,
    epochs=len(graph_history['accuracy']),
    precent=int(graph_history['val_accuracy'][-1]*1000)
))

Сохранение нейронной сети на Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
model.save('gdrive/My Drive/models/function_name_prediction/{name}/maxlen{len}_e{epochs}_p{precent}.h5'.format(
    name=model_name,
    len=maxlen,
    epochs=len(graph_history['accuracy']),
    precent=int(graph_history['val_accuracy'][-1]*1000)
))

Отчистка рабочей директории

In [ ]:
!rm dataset.csv
!rm shuffled_dataset.csv
!rm test_shuffled_dataset.csv
!rm train_shuffled_dataset.csv
!rm dictionary*.json
!rm fnp*.h5